In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from tensorflow import keras
from keras.layers import Input, Dense, LSTM, Conv1D, Bidirectional, Flatten, Concatenate,concatenate,BatchNormalization,MaxPooling1D, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Reshape, normalization
from keras.models import Model
# from keras.utils import to_categorical
import keras.backend as K
# from keras.layers.recurrent import LSTM
from sklearn import metrics
import random
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

2022-06-08 23:34:37.319284: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc


METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

def dnn_model(train_X, train_Y, test_X, test_Y, lr, epoch, batch_size):
    train_X = np.expand_dims(train_X, 2)
    test_X = np.expand_dims(test_X, 2)
    inputs = Input(shape = (train_X.shape[1], train_X.shape[2]))
    x = Conv1D(32, kernel_size = 5, strides = 1, padding = 'same', activation = 'relu')(inputs)
    
    x2 = Conv1D(32, kernel_size=10, strides = 1, padding = 'same', activation = 'relu')(x)
    x = concatenate([inputs, x])
    
    x1 = Conv1D(32, kernel_size=8, strides = 1, padding = 'same', activation = 'relu')(x)
    x1 = concatenate([inputs, x1])
    # x = MaxPooling1D(pool_size = 2, strides = 2, padding = 'same')(x)
    x = Flatten()(x)
    x = Dropout(0.3)(x)
    x1= Flatten()(x1)
    x1= Dropout(0.2)(x1)
    x2 = Flatten()(x2)
    
    x = Dense(32, activation = 'relu')(x)
    x1 = Dense(32)(x1)
    x2=Dense(16)(x2)
    x = concatenate([x, x1])
    # x = Dense(16, activation = 'relu')(x)
    # x = Dropout(0.2)(x)
    x = concatenate([x, x2])
    # x = Dense(8, activation = 'relu')(x)
    predictions = Dense(1, activation = 'sigmoid')(x)
    model = Model(inputs = inputs, outputs = predictions)
    # print(model.summary())
    model.compile(optimizer = 'Adam',
                  loss = 'binary_crossentropy',
                  metrics = METRICS)
    print("compile")
    model.fit(train_X, train_Y, epochs = epoch, batch_size = batch_size, validation_data = (test_X, test_Y), shuffle = True)

    pre_test_y = model.predict(test_X, batch_size = batch_size)
    pre_train_y = model.predict(train_X, batch_size = batch_size)
    test_auc = metrics.roc_auc_score(test_Y, pre_test_y)
    train_auc = metrics.roc_auc_score(train_Y, pre_train_y)
    print("train_auc: ", train_auc)
    print("test_auc: ", test_auc) 
    return test_auc

# Cross Cell Independent Testing

In [8]:

data = np.array(pd.read_csv("6_test.csv"))
pos_number=1658
X1 = data[0:pos_number, 1:]
Y1 = data[0:pos_number, 0]
X2 = data[pos_number:, 1:]
Y2 = data[pos_number:, 0]
X_test = np.concatenate([X1, X2], 0)
Y_test = np.concatenate([Y1, Y2], 0)

print(X1.shape, X2.shape)
print(Y1.shape, Y2.shape)


lr = 0.01
# epoch = 3
batch_size = 32
data = np.array(pd.read_csv("6_train.csv"))
pos_number=2097


X1 = data[0:pos_number, 1:]
Y1 = data[0:pos_number, 0]
X2 = data[pos_number:, 1:]
Y2 = data[pos_number:, 0]
X = np.concatenate([X1, X2], 0)
Y = np.concatenate([Y1, Y2], 0)

(1658, 200) (1656, 200)
(1658,) (1656,)


In [9]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# test_auc = dnn_model(X_train, y_train, X_test, y_test, lr, 5, 32)


test_auc = dnn_model(X, Y, X_test, Y_test, lr, 6, 32)

model
compile
Epoch 1/6
131/131 [==============================] - 2s 7ms/step - loss: 0.3520 - tp: 2746.0000 - fp: 378.0000 - tn: 3374.0000 - fn: 1008.0000 - accuracy: 0.8153 - precision: 0.8790 - recall: 0.7315 - auc: 0.9093 - val_loss: 0.3721 - val_tp: 1238.0000 - val_fp: 163.0000 - val_tn: 1493.0000 - val_fn: 420.0000 - val_accuracy: 0.8241 - val_precision: 0.8837 - val_recall: 0.7467 - val_auc: 0.9251
Epoch 2/6
131/131 [==============================] - 1s 4ms/step - loss: 0.2983 - tp: 1808.0000 - fp: 258.0000 - tn: 1838.0000 - fn: 288.0000 - accuracy: 0.8698 - precision: 0.8751 - recall: 0.8626 - auc: 0.9454 - val_loss: 0.3785 - val_tp: 1234.0000 - val_fp: 205.0000 - val_tn: 1451.0000 - val_fn: 424.0000 - val_accuracy: 0.8102 - val_precision: 0.8575 - val_recall: 0.7443 - val_auc: 0.9170
Epoch 3/6
131/131 [==============================] - 1s 4ms/step - loss: 0.2788 - tp: 1843.0000 - fp: 247.0000 - tn: 1849.0000 - fn: 253.0000 - accuracy: 0.8807 - precision: 0.8818 - recall: 0.87

##### 